In [12]:
import bw2io
from pathlib import Path
import bw2data
from brightpath import SimaproConverter
fp="ESA db/ESA LCA DB v1.1.8 export.CSV"
bw2data.projects.set_current("ecoinvent-3.5-cutoff")
c = SimaproConverter(fp, ecoinvent_version="3.5")

Extracted 817 unallocated datasets in 4.93 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: set_lognormal_loc_value_uncertainty_safe
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: migrate_exchanges
Applying strategy: fix_localized_water_flows
Ap

In [13]:
c.convert_to_brightway()

- format exchanges
- remove empty datasets
- remove empty exchanges
- check inventories
Done!


In [14]:
# if links to EI 35
c.i.data = bw2io.strategies.migrate_exchanges(
    db=c.i.data,
    migration="ESA-ei-3.5"
)

In [16]:
c.i.match_database(fields=["name", "reference product", "location"])
c.i.match_database("ecoinvent-3.5-cutoff", fields=["name", "reference product", "location"])
c.i.match_database("biosphere3", fields=["name", "categories"])

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


In [17]:
c.i.statistics()

822 datasets
159592 exchanges
6841 unlinked exchanges
  Type biosphere: 492 unique unlinked exchanges
  Type technosphere: 15 unique unlinked exchanges


(822, 159592, 6841)

In [6]:
c.i.drop_unlinked(i_am_reckless=True)
if c.i.db_name in bw2data.databases:
    del bw2data.databases[c.i.db_name]
c.i.write_database()

Applying strategy: drop_unlinked
Applied 1 strategies in 0.03 seconds


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Title: Writing activities to SQLite3 database:
  Started: 01/29/2025 14:08:35
  Finished: 01/29/2025 14:08:38
  Total time elapsed: 00:00:02
  CPU %: 99.40
  Memory %: 2.53
Created database: esa lca db v1.1.8 export_edited


Brightway2 SQLiteBackend: esa lca db v1.1.8 export_edited

In [30]:
# this is to build a migration file
migration = {
    "fields": ["name", "reference product", "location", ],
}

migration["data"] = [
    (
        (u["name"], u["reference product"], u["location"], ),
        {}
    )
    for u in c.i.unlinked
    if u["type"] == "technosphere"
]

In [9]:
import json
with open("ESA-ei-3.5_migration.json", 'w') as fp:
    json.dump(migration, fp, indent=2)

In [10]:
with open("ESA-ei-3.5_migration.json") as f_in:
    migration = json.load(f_in)

In [32]:
import pandas as pd
pd.DataFrame.from_dict(c.i.unlinked)[["name", "reference product", "location", "categories", "unit", "simapro name", "type"]].to_excel("unlinked_exchanges.xlsx")